In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from bisect import *
import random

def qm_transfer(reference,uncorrected,value):
    #reference = vector of values representing the percentiles of the reference data
    #uncorrected = vector of values representing the percentiles of the data to be corrected
    #value = Value to be adjusted
    
    #  """Use source and destination percentiles to get the homogenised value.
    ## From Linden Ashcroft's R script (adapted from BoM Python script)
    #Given two lists of values representing, for example,
    #temperatures at the 5th to 95th percentiles, and a value
    #representing an amount to be homogenised, return the
    #homogenised value.
    #"""
    
    if pd.isna(value):
        return(np.nan)
    
    if value < round(uncorrected.iloc[0],3):
        return value + (reference.iloc[0] - uncorrected.iloc[0])
    
        #if the value is less than the lowest percentile, 
        #adjustment is difference between
        #the lowest percentile of the two
    
    if value > round(uncorrected.iloc[-1],3):
        return value + (reference.iloc[-1] - uncorrected.iloc[-1])
    
        #if the value is greater than the highest percentile, 
        #adjustment is difference between
        #the highest percentile of the two
    
    ndx = min(bisect_left(np.array(uncorrected), value), len(uncorrected) - 1)
    
    num_equal = np.count_nonzero(np.array(round(uncorrected,3) == value))
    
    if num_equal == 1:
        return value + (reference.iloc[ndx] - uncorrected.iloc[ndx])
        #If the value is an exact percentile value, 
        #then the adjustment is the difference between that percentile
    elif num_equal > 1:
        offset = random.randint(1, num_equal) - 1
        return value + (reference.iloc[ndx + offset] - uncorrected.iloc[ndx + offset])
    
        #If there are two percentiles with the value, 
        #pick one at random and use that to find the adjustment
        
    else:
        return (((reference.iloc[ndx] - reference.iloc[ndx-1])/
                   (uncorrected.iloc[ndx] - uncorrected.iloc[ndx-1]))*
                  (value - uncorrected.iloc[ndx-1]) + reference.iloc[ndx-1])
    
        #Otherwise, find the percentiles 'bin' that the value fits in, 
        #subtract the Glaisher percentile value from the value of interest
        #multiply that by a ratio of the two percentiles 
        #(if they are changing at the same rate this term will be 1) 
        #and add the Stevenson screen value for that percentile value


In [2]:
inpath = '/home/561/zb8411/Documents/data/observational/perth/temperature/'

historical = pd.read_csv(inpath + 'perthgardens_daily_qc_1880-1900.csv',
                         index_col=0, parse_dates=True, names=['tmax','tmin'], header=0)

modern = pd.read_csv(inpath + 'perthregionaloffice_daily_1897-1992.csv', 
                     index_col=0, parse_dates=True, names=['tmax','tmin'], header=0)

FileNotFoundError: [Errno 2] No such file or directory: '/home/561/zb8411/Documents/data/observational/perth/temperature/perthgardens_daily_qc_1880-1900.csv'

In [ ]:
idx = historical.index.intersection(modern.index)
modern_overlap = modern.loc[idx]
historical_overlap = historical.loc[idx]

In [ ]:
historical

In [ ]:
timestamp = historical.columns
correctedDf = historical.copy()
for col in historical:
    historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05))
    modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
    correctedList = []
    for j in range(len(historical[col])):
        correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j]))
    correctedDf[col + '_corrected'] = correctedList
    del historicalPct, modernPct, correctedList
corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
corrected.columns = corrected.columns.str.replace('_corrected','')
#uncorrected = data.copy()
#data.update(corrected)

In [ ]:
corrected

In [ ]:
historical

In [ ]:
modern = pd.read_csv(inpath + 'perthregionaloffice_daily_1897-1992.csv', 
                     index_col=0, parse_dates=True, names=['tmax','tmin'], header=0)

In [ ]:
f = plt.figure(figsize=(12,12))

ax = f.add_subplot(211)

ax.plot(modern['tmax'].groupby(modern.index.month).mean(),  label = '1897-1992',c='k',ls=':')
ax.plot(corrected['tmax'].groupby(corrected.index.month).mean(),  label = 'Corrected',c='k')
ax.plot(historical['tmax'].groupby(historical.index.month).mean(),  label = 'Uncorrected',c='k',ls='--')

ax.set_ylabel('Maximum daily temperature (C)', fontsize=18)

y = np.arange(1,13,1)
labels = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
ax.set_xticks(y)
ax.set_xticklabels(labels,fontsize=12)
plt.yticks(fontsize=12)
plt.tick_params(labelright=True,right=True)

ax.tick_params(axis='both', which='major', labelsize=14)

ax.text(10.5,26,'A) Tmax',fontweight='bold', fontsize=14)

ax.legend(loc='best', fontsize=12)

ax.set_xlim([1, 12])

ax2 = f.add_subplot(212)

#plt.figure(figsize=(16,7))

ax2.plot(modern['tmin'].groupby(modern.index.month).mean(),  label = '1897-1992',c='k',ls=':')
ax2.plot(corrected['tmin'].groupby(corrected.index.month).mean(),  label = 'Corrected',c='k')
ax2.plot(historical['tmin'].groupby(historical.index.month).mean(),  label = 'Uncorrected',c='k',ls='--')

#plt.plot(concatDf['10:00_corrected'].groupby(concatDf.index.month).mean(),  label = '1843-1862 corrected mean')
#plt.plot(subset['10:00'].groupby(subset.index.month).mean(),  label = '1843-1862 uncorrected mean')

ax2.legend(loc='best', fontsize=12)

#ax2.title.set_text('4pm monthly mean temperatures (corrected/uncorrected)')
ax2.set_xlabel('Month', fontsize=18)
ax2.set_ylabel('Minimum daily temperature (C)', fontsize=18)

ax2.set_xticks(y)
ax2.set_xticklabels(labels,fontsize=14)
plt.yticks(fontsize=12)
plt.tick_params(labelright=True,right=True)

ax2.legend(loc='best', fontsize=14)

ax2.set_xlim([1, 12])
#ax2.text(10.4,29.15,'B) Tmin',fontweight='bold', fontsize=14)

ax2.tick_params(axis='both', which='major', labelsize=14)

f.tight_layout()

f.patch.set_facecolor('white')

#plt.savefig('monthly_mean_temp_pre-post_correction_10am_4pm.tif', dpi=600, format='tif', bbox_inches='tight',facecolor='white',pil_kwargs={"compression": "tiff_lzw"})

In [ ]:
fourpm

In [ ]:
np.arange(1,13,1)

In [ ]:
plt.plot((corrected['tmax'] - historical['tmax']).resample('M').mean())

In [ ]:
modern

In [ ]:
corrected.to_csv(inpath + 'perthgardens_daily_qc_corrected_test_1880-1900.csv')

In [ ]:
f = plt.figure(figsize=(15,5))
plt.plot(corrected['tmax'].loc[idx], label='Corrected')
#plt.plot(modern['tmax'].loc[idx], label='Perth RO')
plt.plot(historical['tmax'].loc[idx], label='Uncorrected')
plt.legend()